In [ ]:
# Setup do Ambiente

# Instalando as dependências necessárias (se não estiverem instaladas)
# !pip install flask
# !pip install flask_sqlalchemy

import sqlite3
from flask import Flask, request, jsonify

# Função para conectar ao banco de dados
def connect_db():
    return sqlite3.connect('nunes_sports.db')

# Criação do Banco de Dados

# Criar a tabela produtos
def criar_tabela():
    conn = connect_db()
    cursor = conn.cursor()

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS produtos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        codigo TEXT NOT NULL UNIQUE,
        descricao TEXT,
        preco REAL NOT NULL
    )
    ''')

    conn.commit()
    conn.close()

criar_tabela()

# Desenvolvimento da API com Flask

app = Flask(__name__)

@app.route('/produtos', methods=['POST'])
def criar_produto():
    data = request.json
    nome = data.get('nome')
    codigo = data.get('codigo')
    descricao = data.get('descricao')
    preco = data.get('preco')

    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('''
    INSERT INTO produtos (nome, codigo, descricao, preco)
    VALUES (?, ?, ?, ?)
    ''', (nome, codigo, descricao, preco))
    conn.commit()
    conn.close()

    return jsonify({"message": "Produto criado com sucesso!"}), 201

@app.route('/produtos', methods=['GET'])
def listar_produtos():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM produtos')
    produtos = cursor.fetchall()
    conn.close()

    produtos_list = []
    for produto in produtos:
        produtos_list.append({
            "id": produto[0],
            "nome": produto[1],
            "codigo": produto[2],
            "descricao": produto[3],
            "preco": produto[4]
        })

    return jsonify(produtos_list)

@app.route('/produtos/<int:id>', methods=['PUT'])
def atualizar_produto(id):
    data = request.json
    nome = data.get('nome')
    codigo = data.get('codigo')
    descricao = data.get('descricao')
    preco = data.get('preco')

    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('''
    UPDATE produtos SET nome = ?, codigo = ?, descricao = ?, preco = ?
    WHERE id = ?
    ''', (nome, codigo, descricao, preco, id))
    conn.commit()
    conn.close()

    return jsonify({"message": "Produto atualizado com sucesso!"})

@app.route('/produtos/<int:id>', methods=['DELETE'])
def deletar_produto(id):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('DELETE FROM produtos WHERE id = ?', (id,))
    conn.commit()
    conn.close()

    return jsonify({"message": "Produto deletado com sucesso!"})

# Frontend Simples

html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>CRUD de Produtos</title>
</head>
<body>
    <h1>CRUD de Produtos</h1>
    
    <div id="produtos">
        <form id="formProduto">
            <input type="text" id="nome" placeholder="Nome" required>
            <input type="text" id="codigo" placeholder="Código" required>
            <input type="text" id="descricao" placeholder="Descrição">
            <input type="number" id="preco" placeholder="Preço" required>
            <button type="submit">Salvar</button>
        </form>
        
        <table id="tabelaProdutos">
            <thead>
                <tr>
                    <th>Nome</th>
                    <th>Código</th>
                    <th>Descrição</th>
                    <th>Preço</th>
                    <th>Ações</th>
                </tr>
            </thead>
            <tbody></tbody>
        </table>
    </div>

    <script>
        function carregarProdutos() {
            fetch('/produtos')
                .then(response => response.json())
                .then(data => {
                    const tabela = document.getElementById('tabelaProdutos').getElementsByTagName('tbody')[0];
                    tabela.innerHTML = '';

                    data.forEach(produto => {
                        const row = tabela.insertRow();
                        row.insertCell(0).innerText = produto.nome;
                        row.insertCell(1).innerText = produto.codigo;
                        row.insertCell(2).innerText = produto.descricao;
                        row.insertCell(3).innerText = produto.preco;
                        
                        const acoesCell = row.insertCell(4);
                        acoesCell.innerHTML = `
                            <button onclick="editarProduto(${produto.id})">Editar</button>
                            <button onclick="deletarProduto(${produto.id})">Deletar</button>
                        `;
                    });
                });
        }

        document.getElementById('formProduto').onsubmit = function(e) {
            e.preventDefault();

            const nome = document.getElementById('nome').value;
            const codigo = document.getElementById('codigo').value;
            const descricao = document.getElementById('descricao').value;
            const preco = document.getElementById('preco').value;

            fetch('/produtos', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({ nome, codigo, descricao, preco })
            }).then(response => {
                if (response.ok) {
                    carregarProdutos();
                }
            });
        };

        function editarProduto(id) {
            // Implementar lógica para editar produtos
        }

        function deletarProduto(id) {
            fetch(`/produtos/${id}`, {
                method: 'DELETE'
            }).then(response => {
                if (response.ok) {
                    carregarProdutos();
                }
            });
        }

        carregarProdutos();
    </script>
</body>
</html>
"""

from IPython.core.display import display, HTML
display(HTML(html_code))

# Execução do Flask
app.run()


C:\Users\Everton Reinaldo\AppData\Local\Temp\ipykernel_2180\2536607749.py:209: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Nome,Código,Descrição,Preço,Ações


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
